# LDO design and documentation

To start: let's estimate some rough specs needed for the design (overestimate, honestly):

- Iout ~ 1mA
- Cout ~ 10p (estimated as 100u / 10u x 10 -- really the max bound is what matters for setting secondary(?) pole)
- Vin ~ 3.3-6V
- Vout = 3V
- Vref = 0.5V

Assume that we can give like 0.2 x 0.2mm to the LDO. so a max cap of 200u x 200u -> modeled as mimcap w/:

- C = 80p -> might need to be higher. maybe target ~150p? and compromise on space as needed
- R_esr ??? -- intuitively, this will be increasingly small. is it important to introduce an ESR for stability? The take is no, especially with a "capless" design. An extra resistance can be introduced in the miller feedback path in a multistage design

A few other things to consider adding long-term:

- Soft-start (avoid straining the digital core during startup bc/ inrush. although, we shouldn't expect a lot of inrush current to load things down)
- Add a buffer to increase PSRR
- A capless/on-chip LDO (like this one) requires extremely high gain bandwidth, with the dominant pole being set by millering the first stage (7.10/Fig 7.20, Design of PMIC)

With these specs in mind, we can solve for values to estimate where the poles and zeroes land.

In [2]:
import math
from sympy import *

In [ ]:
# (7.63), assuming a buffered capacitor-less LDO with miller feedback and a compensation resistor
# assume that we're a bit underconstrainted.
I_passfet = 1e-3 # target 1mA
V_th_pmos = 0.85 # roughly
V_gs_passfet = 5-1.5 # assume driven at a 3v3 midpoint, so like 1.5V input
V_eff_passfet = V_gs_passfet - V_th_pmos
gm_passfet = 2*I_passfet / (V_eff_passfet) # ~ 0.7 -- checks out with OP analysis

# backcalc lambda from an OP -- assume min length device = 0.55u
_ron = 96e3
lambda_pmos = 1 / (_ron * 10.4e-6) # suspiciously close to 1 but whatever
rds_passfet = 1 / (lambda_pmos * I_passfet)

# guess at error amplifier output resistance by assuming it's a parallel cascode struct
# rout = gm ro^2 = u ro, where u is a device param
# assume we're using near min-sized devices, sim to find values. calculated with 3u/min length devices
# expect these to be around 10-100 (really ~3-4. not great but whatever.)
u_nmos = 22e3 * 0.15e-3
u_pmos = 96e3 * 50e-6
print(u_nmos, u_pmos)
r_nmos_2u = 22e3
r_pmos_2u = 96e3
# guess at rout for amplifier, assume cascode
_a = r_nmos_2u * u_nmos
_b = r_pmos_2u * u_pmos
r_erramp_out = _a*_b / (_a+_b)
print(r_erramp_out) # experimentally should be like 70k

# eventually we want to constrain to find a GBW target -- so it's two dimensional with A2 and C_compensation
A2, C_compensation, GBW, freq_pole = symbols('A2 C_compensation GBW freq_pole')

e2 = Eq(GBW, A2 * freq_pole)

# and, because things were breaking, assume that gm*rds for the passfet is roughly u_pmos
C1 = A2 * u_pmos * C_compensation
e1 = Eq(2*math.pi*freq_pole, 1/(r_erramp_out * C1))

print(solve([e1, e2], [A2, C_compensation, GBW, freq_pole])) # gives gain in terms of c_compensation

# which gives a C_comp in terms of GBW
cc_rel = 5.2866e-7 # F / unit GBW

# and working the other way, assume we have like a 1pF cap for compensation...
cc = 1e-12
print(cc * (1/cc_rel)) #?????? why like 1.8u for GBW. something's off.

3.3 4.8
62718.5601799775
[(GBW/freq_pole, 5.28667745491337e-7/GBW, GBW, freq_pole)]
1.8915749252827904e-06


In [48]:
gm_passfet*rds_passfet

0.7535094339622643